In [76]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.stats import poisson, uniform, expon, pareto
from scipy.optimize import minimize, fsolve
import scipy.optimize as opt
import scipy.integrate as integrate
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex
from scipy.optimize import least_squares

# tullock

In [67]:
n = 2

In [68]:
bs = [sp.Symbol('b{}'.format(i), positive=True, real=True) for i in range(n)]
vs = [sp.Symbol('v{}'.format(i), positive=True, real=True) for i in range(n)]

den = sum(bs)
xs  = [b/den for b in bs]
us  = [xs[i]*vs[i] - bs[i] for i in range(n)] 

In [69]:
us[0]

b0*v0/(b0 + b1) - b0

In [70]:
focs = [sp.Eq(sp.diff(us[i], bs[i]),0) for i in range(n)]
focs[0]

Eq(-b0*v0/(b0 + b1)**2 + v0/(b0 + b1) - 1, 0)

In [71]:
focs[1]

Eq(-b1*v1/(b0 + b1)**2 + v1/(b0 + b1) - 1, 0)

In [72]:
solutions = sp.solve(focs, bs, dict=True)

In [73]:
solutions[0]

{b0: v0*v1*(v0**2 + 2*v0*v1 + v1**2 - v1*sqrt(v0**2 + 2*v0*v1 + v1**2))/(v0**2 + 2*v0*v1 + v1**2)**(3/2),
 b1: v0*v1**2/(v0**2 + 2*v0*v1 + v1**2)}

In [75]:
xhat = [x.subs(solutions[0]) for x in xs]
sp.simplify(xhat[1])

v1/sqrt(v0**2 + 2*v0*v1 + v1**2)

$$
    x_0 = \frac{v_0}{v_0+v_1}, \quad x_1 = \frac{v_1}{v_0+v_1}
$$

In [83]:
[xhat[i].subs({vs[0]: 3, vs[1]: 4}) for i in range(n)]

[3/7, 4/7]

In [100]:
values = [3, 400]

def U_i(x, i):
    return values[i] * x

def U_tilde(x):
    """
    Compute the total modified utility:
      sum_i [(1-x_i)*U_i(x_i) + \int_0^{x_i} U_i(y) dy]
    where x is a vector of allocations.
    """
    total = 0
    for i in range(n):
        xi = x[i]
        term1 = (1 - xi) * U_i(xi, i)
        term2, err = integrate.quad(lambda y: U_i(y, i), 0, xi)
        total += term1 + term2
    return total

def objective(x):
    return -U_tilde(x)

constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
bounds = [(0, 1) for _ in range(n)]
x0 = np.array([1.0/n] * n)

result = opt.minimize(objective, x0, bounds=bounds, constraints=constraints)

In [101]:
3/403, 400/403

(0.007444168734491315, 0.9925558312655087)

In [102]:
# Print the results.
print("Optimal allocations:", result.x)
print("Maximized total modified utility:", -result.fun)

Optimal allocations: [0.00744416 0.99255584]
Maximized total modified utility: 200.0111662531017


In [103]:
U_i(result.x[0], 0), U_i(result.x[1], 1)

(0.022332493782350967, 397.0223341623532)